In [4]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [54]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = GPT2Model.from_pretrained('gpt2-xl')

In [7]:
# sparsify at 10%, 50%, 90%, 95%, 99%

In [29]:
model.named_parameters

<bound method Module.named_parameters of GPT2Model(
  (wte): Embedding(50257, 1600)
  (wpe): Embedding(1024, 1600)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (

In [ ]:
# test

In [42]:
params = model.state_dict() 
#params.keys()

In [51]:
def prune_gpt2_layers(ratio):

    for i in range(48):
        list= ['h.{}.attn.c_attn'.format(i),
                'h.{}.attn.c_proj'.format(i),
                #'h.{}.ln_1'.format(i), leave layer norms unpruned
                #'h.{}.ln_2'.format(i),
                'h.{}.mlp.c_fc'.format(i),
                'h.{}.mlp.c_proj'.format(i)]
        for name, module in model.named_modules():
            if name in list:
                prune.l1_unstructured(module, name='weight', amount=ratio)
                prune.remove(module, 'weight')
                prune.l1_unstructured(module, name='bias', amount=ratio)
                prune.remove(module, 'bias')

In [41]:
prune_gpt2_layers(0.2)

In [43]:
params['h.1.attn.c_attn.weight'].nelement()

7680000

In [44]:
print(
    "Sparsity in h.1.attn.c_attn.weight: {:.2f}%".format(
        100. * float(torch.sum(params['h.1.attn.c_attn.weight'] == 0))
        / float(params['h.1.attn.c_attn.weight'].nelement())
    )
)

Sparsity in h.1.attn.c_attn.weight: 20.00%


In [49]:
prune_gpt2_layers(0.9)

In [50]:
print(
    "Sparsity in h.1.attn.c_attn.weight: {:.2f}%".format(
        100. * float(torch.sum(params['h.1.attn.c_attn.weight'] == 0))
        / float(params['h.1.attn.c_attn.weight'].nelement())
    )
)

Sparsity in h.1.attn.c_attn.weight: 20.00%


In [48]:
params['h.1.attn.c_attn.weight']

tensor([[ 0.0264,  0.0000, -0.0758,  ...,  0.0295, -0.0168,  0.0607],
        [ 0.0830,  0.0267, -0.0111,  ...,  0.0275,  0.0240,  0.0281],
        [ 0.0480, -0.0213, -0.0625,  ...,  0.0000,  0.0236, -0.0187],
        ...,
        [ 0.0390, -0.0000,  0.0541,  ...,  0.0582, -0.0000,  0.0000],
        [-0.0153, -0.0084,  0.0161,  ...,  0.0000,  0.0089, -0.0000],
        [ 0.0593,  0.0978, -0.0552,  ...,  0.0217, -0.0143,  0.0101]])

In [18]:
params = model.state_dict() 
params.keys()

odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])

In [56]:
print(
    "Sparsity in h.{}.attn.c_attn.weight: {:.2f}%".format(10,
        100. * float(torch.sum(params[f'h.{10}.attn.c_attn.weight'] == 0))
        / float(params[f'h.10.attn.c_attn.weight'].nelement())
    )
)

Sparsity in h.10.attn.c_attn.weight: 20.00%


In [61]:
sum(sum(params['h.10.attn.c_attn.weight'] == 0))

tensor(1536000)

In [63]:
params['h.10.attn.c_attn.weight'].nelement()

7680000

In [64]:
sum(sum(params['h.10.attn.c_attn.weight'] == 0)) / params['h.10.attn.c_attn.weight'].nelement()

tensor(0.2000)

In [65]:
prune_gpt2_layers(0.5)

In [81]:
sum(sum(model.h[10].attn.c_attn.weight == 0)) / model.h[10].attn.c_attn.weight.nelement()

tensor(0.5000)

In [72]:
prune.l1_unstructured(model.h.10.attn.c_attn, name='weight', amount=ratio)
                prune.remove(module, 'weight')

SyntaxError: invalid syntax (<ipython-input-72-99beac4568fd>, line 1)

In [75]:
model.named_modules

<bound method Module.named_modules of GPT2Model(
  (wte): Embedding(50257, 1600)
  (wpe): Embedding(1024, 1600)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_

In [79]:
model.h[10].attn.c_attn.weight

Parameter containing:
tensor([[ 0.0000, -0.0577, -0.0430,  ..., -0.0271, -0.0000, -0.0463],
        [-0.0701,  0.0000, -0.0299,  ..., -0.0358,  0.0000, -0.0000],
        [ 0.0590,  0.0287, -0.0000,  ..., -0.0000, -0.0000,  0.0347],
        ...,
        [-0.0852,  0.0000,  0.0000,  ...,  0.0338,  0.0000,  0.0000],
        [ 0.0000, -0.0000, -0.0302,  ..., -0.0350, -0.0383, -0.0000],
        [ 0.0000,  0.0000,  0.0344,  ..., -0.0000,  0.0522, -0.0000]],
       requires_grad=True)

In [80]:
params['h.10.attn.c_attn.weight']

tensor([[ 0.0000, -0.0577, -0.0430,  ..., -0.0271, -0.0114, -0.0463],
        [-0.0701,  0.0133, -0.0299,  ..., -0.0358,  0.0244, -0.0000],
        [ 0.0590,  0.0287, -0.0167,  ..., -0.0103, -0.0111,  0.0347],
        ...,
        [-0.0852,  0.0225,  0.0196,  ...,  0.0338,  0.0103,  0.0000],
        [ 0.0124, -0.0000, -0.0302,  ..., -0.0350, -0.0383, -0.0209],
        [ 0.0118,  0.0000,  0.0344,  ..., -0.0099,  0.0522, -0.0143]])